In [ ]:
from pathlib import Path
import shutil
import requests
import zipfile
import rasterio
import os
import numpy as np
from rasterio.warp import reproject, Resampling
import geopandas as gpd
from raster_manipulation import save_raster, clip_raster, exact_raster_reprojection, fast_raster_reprojection
import wpgpDownload
from wpgpDownload.utils.convenience_functions import download_country_covariates as download_worldpop

# Ignore warnings from numpy (division by 0 or nan)
np.seterr(divide='ignore', invalid='ignore')

# Downloading population data

Correcting NTL data requires population datasets. The most suited dataset is HRSL but it is not available for all countries. WorldPop data remain a good alternative.

In [ ]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset

setup_logging()
Configuration.create(hdx_site='prod', user_agent='Read-only user', hdx_read_only=True)

In [ ]:
from wpgpDownload.utils.convenience_functions import refresh_csv
refresh_csv()

In [ ]:
hdx_datasets = Dataset.search_in_hdx('hrsl', rows=500)
hdx_resources = Dataset.get_all_resources(hdx_datasets)

country_names = [f.name for f in Path("./").iterdir() if len(f.name)==3]

for country in country_names:
    print(country)

    for res in hdx_resources:
        if 'population_'+country.lower() in res['name'] and '.zip' in res['name'] and 'csv' not in res['name']:
            print('Downloading HRSL',res['name'], end='\r')
            url, path = res.download()
            print('HRSL',res['name'],'download completed       ')
            shutil.move(Path(path),Path('./'+country+'/misc_data/population_'+country.lower()+'.zip'))
            zipfile.ZipFile(Path('./'+country+'/misc_data/population_'+country.lower()+'.zip'), 'r').extractall(Path('./'+country+'/misc_data'))
            for file in Path('./'+country+'/misc_data').iterdir():
                if 'population_'+country.lower() in file.name and file.suffix != '.tif':
                    os.remove(file)

    years = [2017] # WorldPop 2017 will be used alongside HRSL (if available) to detect inhabited areas

    #NTL_files = [file for file in Path("./"+country+"/NTL").iterdir() if "NTL" in file.name]
    #
    #years = []
    #for NTL_file in NTL_files:
    #    years.append(NTL_file.name[4:8])
    #years = [year for year in set(years)]
    #years.sort()

    for year in years:
        print('Downloading WorldPop '+country+' '+str(year)+'\t\t',end='\r')
        download_worldpop(ISO=country,out_folder='.\\'+country+'\\worldpop',prod_name='ppp_'+str(year))
        print('WorldPop '+country+' '+str(year)+' download completed\t\t')
    
    print("")
    
print('Done')

# Population raster reprojection

In [ ]:
country_names = [f.name for f in Path("./").iterdir() if len(f.name)==3]

for country in country_names:
    print('')
    print(country)
    
    # Clipping all NTL rasters according to GADM boundaries
    
    path_in = Path("./"+country)
    
    try:
        aoi_name = "gadm36_"+country+".gpkg"
        aoi_in = Path("./"+country+"/misc_data") / aoi_name
        aoi = gpd.read_file(aoi_in, layer='gadm36_'+country+'_0')
        
    except:
        print('Downloading gadm36_'+country+'_gpkg.zip')
        url = 'https://biogeo.ucdavis.edu/data/gadm3.6/gpkg/gadm36_'+country+'_gpkg.zip'
        r = requests.get(url, allow_redirects=True)
        open('./'+country+'/misc_data/gadm36_'+country+'_gpkg.zip', 'wb').write(r.content)
        zipfile.ZipFile(Path('./'+country+'/misc_data/gadm36_'+country+'_gpkg.zip'), 'r').extractall(Path('./'+country+'/misc_data'))
        os.remove(Path('./'+country+'/misc_data/gadm36_'+country+'_gpkg.zip'))
        os.remove(Path('./'+country+'/misc_data/license.txt'))        
        print('gadm36_'+country+'.gpkg download completed')
        
        aoi_name = "gadm36_"+country+".gpkg"
        aoi_in = Path("./"+country+"/misc_data") / aoi_name
        aoi = gpd.read_file(aoi_in, layer='gadm36_'+country+'_0')
    
    for NTL_file in Path("./"+country+"/NTL").iterdir():
        clipped, aff, crs = clip_raster(NTL_file, aoi)
        save_raster(NTL_file, clipped, aff)
        print(NTL_file.name, 'clipped')
    
    # Opening last NTL raster to get the targeted resolution
    
    ntl_in = Path("./"+country+"/NTL") / NTL_file.name
    ntl = rasterio.open(ntl_in)
    
    # Reprojecting HRSL raster

    misc_data_files = [file.name for file in Path("./"+country+"/misc_data").iterdir()]
    if ("HRSL_"+country+"_NTL-resolution.tif" not in misc_data_files):
        if len([file for file in misc_data_files if 'population_'+country.lower() in file])==1:
            hrsl_name = [file for file in misc_data_files if 'population_'+country.lower() in file][0]
            hrsl_in = Path("./"+country+"/misc_data") / hrsl_name
            hrsl = rasterio.open(hrsl_in)
            
            print('Reprojecting ', hrsl_name)
            hrsl_ntl_reso, _ = exact_raster_reprojection(hrsl, ntl)

            ntl_out = "HRSL_"+country+"_NTL-resolution.tif"
            path_out = Path("./"+country+"/misc_data") / ntl_out
            save_raster(path_out, hrsl_ntl_reso, ntl.transform)
            clipped, aff, crs = clip_raster(path_out, aoi)
            save_raster(path_out, clipped, aff)
            
        else:
            print('Unable to find one HRSL file')
                
    # Reprojecting WorldPop rasters
    
    worldpop_files = [file for file in Path("./"+country+"/worldpop").iterdir() if 'ppp' in file.name]
    if len(worldpop_files) > 0:
        for file in worldpop_files:
            if "worldpop_"+file.stem[-4:]+'_'+country+"_NTL-resolution.tif" not in [file.name for file in Path("./"+country+"/worldpop").iterdir()]:
                worldpop_in = Path("./"+country+"/worldpop") / file.name
                worldpop = rasterio.open(worldpop_in)
                worldpop_rd = worldpop.read(1)

                print('Reprojecting ',file.name)
                #worldpop_ntl_reso, _ = exact_raster_reprojection(worldpop, ntl) # For exact but slow reprojection
                worldpop_ntl_reso = fast_raster_reprojection(worldpop, ntl)

                ntl_out = "worldpop_"+file.stem[-4:]+'_'+country+"_NTL-resolution.tif"
                path_out = Path("./"+country+"/worldpop") / ntl_out
                save_raster(path_out, worldpop_ntl_reso, ntl.transform)
                clipped, aff, crs = clip_raster(path_out, aoi)
                save_raster(path_out, clipped, aff)
    else:
        print('Unable to find a WorldPop file')
        
print('')
print('Done')